<a href="https://colab.research.google.com/github/i-SanMartin/FastAI/blob/main/04_MNIST_Basics/Basic_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#hide
!pip install -Uqq fastbook fastai==2.2.5
import fastbook
fastbook.setup_book()

In [22]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [23]:
#Download data from url to default dest.
#Set path as the BASE_PATH (fastai thing) and look for the elements inside
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path
path.ls()

(#3) [Path('labels.csv'),Path('train'),Path('valid')]

In [24]:
#Look for the elements in train.
(path/'train').ls()

(#2) [Path('train/3'),Path('train/7')]

In [25]:
#Save the elements in the folders 3 and 7.
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

#Open images as tensors.
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]

#Stack tensors.
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255
stacked_threes.shape

torch.Size([6131, 28, 28])

In [26]:
#Define train values.
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)

In [27]:
#Define train labels. Label 1 for threes and 0 for sevens
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
train_x.shape,train_y.shape

(torch.Size([12396, 784]), torch.Size([12396, 1]))

In [28]:
#Create DataSet
dset = list(zip(train_x,train_y))
x,y = dset[0]
x.shape,y

(torch.Size([784]), tensor([1]))

In [29]:
#Stack validation tensors.
valid_3_tens = torch.stack([tensor(Image.open(o)) 
                            for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o)) 
                            for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
valid_3_tens.shape,valid_7_tens.shape

(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [30]:
#Define validation values labels.
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [31]:
#Function to generate weigths.
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [39]:
#Generate weights.
weights = init_params((28*28,1))
weights.shape

torch.Size([784, 1])

In [41]:
#Generate bias.
bias = init_params(1)
bias.shape

torch.Size([1])

In [42]:
#One prediction.
pred = train_x[0]@weights + bias
pred

tensor([9.0484], grad_fn=<AddBackward0>)

In [45]:
#All predictions.
def linear1(xb): return xb@weights + bias
pred = torch.sigmoid(linear1(train_x))
pred

tensor([[9.9988e-01],
        [9.9956e-01],
        [1.0000e+00],
        ...,
        [9.4722e-03],
        [9.9931e-01],
        [1.6603e-04]], grad_fn=<SigmoidBackward>)

In [62]:
#Correct predictions.
corrects = (pred > 0.5) == train_y
corrects.float().mean().item()

0.6352049112319946

In [63]:
#Calculate loss.
def loss_func(results, label): return torch.where(label==1, 1 - pred, pred).mean()

loss = loss_func(pred, train_y)
loss

tensor(0.3712, grad_fn=<MeanBackward0>)